In [ ]:
import pandas as pd
import numpy as np

Importando Banco de Dados

In [ ]:
df = pd.read_csv('train.csv', index_col='id')

In [ ]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
id,,,,,,,,,,,,,,,,,,,,,
0,52,technician,divorced,professional.course,NaN,yes,no,telephone,jun,mon,56,8,999,0,nonexistent,1.4,94.465,-41.8,4.865,5228.1,no
1,36,management,married,university.degree,no,yes,no,cellular,nov,tue,267,2,999,0,nonexistent,-0.1,93.200,-42.0,4.153,5195.8,no
2,53,blue-collar,married,high.school,no,yes,yes,cellular,jul,mon,243,2,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
3,45,blue-collar,single,basic.9y,no,yes,no,cellular,aug,wed,249,1,4,1,success,-2.9,92.201,-31.4,0.879,5076.2,no
4,60,housemaid,married,high.school,NaN,yes,no,telephone,may,wed,74,1,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,no


In [ ]:
df = df.dropna()

In [ ]:
for c in df.columns[df.dtypes == object]:
    df[c] = df[c].astype('category')

In [ ]:
df.education.cat.reorder_categories(['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 
       'professional.course', 'university.degree'], ordered=True, inplace=True)

In [ ]:
for c in df.columns[df.dtypes == 'category']:
    df[c] = df[c].cat.codes

In [ ]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
id,,,,,,,,,,,,,,,,,,,,,
1,36,4,1,6,0,1,0,0,7,3,267,2,999,0,1,-0.1,93.200,-42.0,4.153,5195.8,0
2,53,1,1,4,0,1,1,0,3,1,243,2,999,0,1,1.4,93.918,-42.7,4.962,5228.1,0
3,45,1,2,3,0,1,0,0,1,4,249,1,4,1,2,-2.9,92.201,-31.4,0.879,5076.2,0
6,48,9,2,5,0,1,0,0,6,4,142,2,999,0,1,-1.8,92.893,-46.2,1.334,5099.1,0
13,34,0,1,6,0,1,0,0,6,3,88,3,999,1,0,-1.8,92.893,-46.2,1.344,5099.1,0


In [ ]:
#previsores = df.drop(['default','contact', 'duration', 'previous', 'campaign', 'pdays', 'y'],  axis=1)
previsores = df.drop(['y'],  axis=1)
previsores.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
id,,,,,,,,,,,,,,,,,,,,
1,36,4,1,6,0,1,0,0,7,3,267,2,999,0,1,-0.1,93.200,-42.0,4.153,5195.8
2,53,1,1,4,0,1,1,0,3,1,243,2,999,0,1,1.4,93.918,-42.7,4.962,5228.1
3,45,1,2,3,0,1,0,0,1,4,249,1,4,1,2,-2.9,92.201,-31.4,0.879,5076.2
6,48,9,2,5,0,1,0,0,6,4,142,2,999,0,1,-1.8,92.893,-46.2,1.334,5099.1
13,34,0,1,6,0,1,0,0,6,3,88,3,999,1,0,-1.8,92.893,-46.2,1.344,5099.1


In [ ]:
classe = df.filter('y')
classe.head()

,y
id,
1,0
2,0
3,0
6,0
13,0


In [ ]:
sum(classe['y'])

3463

In [ ]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#previsores = scaler.fit_transform(previsores)
#classe = scaler.fit_transform(classe)

In [ ]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.15, random_state=42, shuffle=True, stratify=classe)

In [ ]:
previsores_treinamento.shape, previsores_teste.shape, classe_treinamento.shape, classe_teste.shape

((23280, 20), (4109, 20), (23280, 1), (4109, 1))

Análise bayesiana

In [ ]:
from sklearn.naive_bayes import GaussianNB
regressor = GaussianNB()
regressor.fit(previsores_treinamento, classe_treinamento)

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
regressor.score(previsores_treinamento, classe_treinamento)

0.8246134020618556

In [ ]:
y_pred = regressor.predict(previsores_teste)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(classe_teste, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.85      0.89      3589
           1       0.38      0.62      0.47       520

    accuracy                           0.82      4109
   macro avg       0.66      0.73      0.68      4109
weighted avg       0.87      0.82      0.84      4109



Análise K-means

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_class = KNeighborsClassifier(algorithm='auto', metric='minkowski', leaf_size=30, n_neighbors=5)
knn_class.fit(previsores_treinamento, classe_treinamento)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
knn_class.score(previsores_treinamento, classe_treinamento)

0.922938144329897

In [ ]:
y_pred = knn_class.predict(previsores_teste)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(classe_teste, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3589
           1       0.57      0.48      0.52       520

    accuracy                           0.89      4109
   macro avg       0.75      0.71      0.73      4109
weighted avg       0.88      0.89      0.88      4109



In [ ]:
sum(y_pred)

434

RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
regressor_rf = RandomForestClassifier(n_estimators=10000, criterion='entropy', random_state=42, max_depth=1000)
regressor_rf.fit(previsores_treinamento, classe_treinamento)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=1000, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
regressor_rf.score(previsores_treinamento, classe_treinamento)

1.0

In [ ]:
y_pred = regressor_rf.predict(previsores_teste)

In [ ]:
print(metrics.classification_report(classe_teste, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.95      3589
           1       0.66      0.54      0.59       520

    accuracy                           0.91      4109
   macro avg       0.80      0.75      0.77      4109
weighted avg       0.90      0.91      0.90      4109



Análise SVM

In [ ]:
from sklearn.svm import SVC
regressor = SVC(kernel='rbf', random_state=1, C=2)
regressor.fit(previsores_treinamento, classe_treinamento)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=2, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
regressor.score(previsores_treinamento, classe_treinamento)

0.8861254295532646

In [ ]:
y_pred = regressor.predict(previsores_teste)

In [ ]:
print(metrics.classification_report(classe_teste, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      3589
           1       0.62      0.22      0.33       520

    accuracy                           0.88      4109
   macro avg       0.76      0.60      0.63      4109
weighted avg       0.86      0.88      0.86      4109



In [ ]:
#!pip install tensorflow
#!pip install keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classificador = Sequential( )
classificador.add(Dense(units=40, activation='relu', input_dim=20), )
classificador.add(Dense(units=40, activation='relu'))
model.add(Dropout(0.2))
classificador.add(Dense(units=40, activation='relu'))
model.add(Dropout(0.2))
classificador.add(Dense(units=40, activation='relu'))
model.add(Dropout(0.2))
classificador.add(Dense(units=40, activation='relu'))
classificador.add(Dense(units=1, activation='sigmoid'))
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classificador.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 40)                840       
_________________________________________________________________
dense_29 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_30 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_31 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_32 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 41        
Total params: 7,441
Trainable params: 7,441
Non-trainable params: 0
___________________________________________________

In [ ]:
classificador.fit(previsores_treinamento, classe_treinamento, batch_size=10, epochs=100)

In [ ]:
y_pred = classificador.predict(previsores_teste)

In [ ]:
#lista = lis
#for x in y_pred:
 #   print(x)

In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
print(metrics.classification_report(classe_teste, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      3589
           1       0.62      0.46      0.53       520

    accuracy                           0.90      4109
   macro avg       0.77      0.71      0.73      4109
weighted avg       0.89      0.90      0.89      4109



Kadle

In [ ]:
aval = pd.read_csv('test.csv', index_col='id')
sub = pd.read_csv('sample.csv', index_col='id')

In [ ]:
#aval.head()

In [ ]:
for c in aval.columns[aval.dtypes == object]:
    aval[c] = aval[c].astype('category')
aval.education.cat.reorder_categories(['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 
       'professional.course', 'university.degree'], ordered=True, inplace=True)
for c in aval.columns[aval.dtypes == 'category']:
    aval[c] = aval[c].cat.codes

In [ ]:
#aval.head()

In [ ]:
#previsores = aval.drop(['contact', 'duration', 'previous'],  axis=1)

In [ ]:
#aval = scaler.fit_transform(aval)

In [ ]:
y_aval = regressor_rf.predict(aval)

In [ ]:
#y_aval = (y_aval > 0.1)

In [ ]:
y_aval

array([0, 1, 1, ..., 1, 0, 0], dtype=int8)

In [ ]:
sub.y = pd.Series(y_aval)

In [ ]:
sub.head()

,y
id,
0,0
1,1
2,1
3,0
4,0


In [ ]:
sub.to_csv('Resultado_lucas_rf.csv')